In [1]:
!python --version

Python 3.7.4


In [2]:
# !pip install -q -U pip
# !pip install -q numpy
# !pip install -q pandas
# !pip install -q ckiptagger
# !pip install -q tqdm
# !pip install -q tensorflow==1.14.0
# !pip install -q ipywidgets
# !pip install -q matplotlib

In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [6]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [7]:
ws = WS('./data/')
pos = POS('./data/')

In [8]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [9]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

# 尋找降維的詞向量：PPMI + SVD

In [10]:
word2index = {}
index2word = {}
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1

In [11]:
len(word2index)

6690

如果使用one-hot就需要這麼大的維度的詞向量

In [13]:
# 建立Co-Matrix

vocab_size = len(word2index)
co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)

window_size = 2
# YOUR CODE HERE

for index in train_title_cuts:
    for i in range(len(train_title_cuts[index])-window_size):
        word = train_title_cuts[index][i][0]
        word_next = [ train_title_cuts[index][i+j+1][0] for j in range(window_size)]
        for w in word_next:
            co_matrix[word2index[word], word2index[w]] += 1
            co_matrix[word2index[w], word2index[word]] += 1


# END YOUR CODE

In [14]:
co_matrix

array([[0, 2, 1, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
#co_matrix

array([[0, 2, 0, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [15]:
co_matrix.shape

(6690, 6690)

In [16]:
# 建立PPMI

def get_ppmi(co_matrix: np.ndarray, eps: float=1e-8):
    # YOUR CODE HERE
    N = np.sum(co_matrix)
    S = np.sum(co_matrix, axis=0)
    M = np.zeros(shape=co_matrix.shape)
    for row in range(co_matrix.shape[0]):
        for column in range(co_matrix.shape[1]):
            pmi = np.log2(co_matrix[row, column]*N/(S[row]*S[column])+eps)
            M[row, column] = max(0, pmi)

    # END YOUR CODE
    return M

ppmi = get_ppmi(co_matrix)

In [17]:
ppmi

array([[ 0.        ,  8.49278787,  9.39967847, ...,  0.        ,
         0.        ,  0.        ],
       [ 8.49278787,  0.        , 10.49278787, ...,  0.        ,
         0.        ,  0.        ],
       [ 9.39967847, 10.49278787,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
#ppmi

array([[ 0.      ,  9.267157,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 9.267157,  0.      , 11.515084, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      , 11.515084,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [18]:
# 進行SVD分解，並得到降維的詞向量

from sklearn.decomposition import TruncatedSVD

# 使用`TruncatedSVD`進行降維，降維到dim=1000
# YOUR CODE HERE

svd1 = TruncatedSVD(n_components=1000)
word_vectors1 = svd1.fit_transform(ppmi)
svd2 = TruncatedSVD(n_components=2000)
word_vectors2 = svd2.fit_transform(ppmi)
svd3 = TruncatedSVD(n_components=3000)
word_vectors3 = svd3.fit_transform(ppmi)
print('Reduce dimension to 1000 and explained variance = ', svd1.explained_variance_ratio_.sum())
print('Reduce dimension to 2000 and explained variance = ', svd2.explained_variance_ratio_.sum())
print('Reduce dimension to 3000 and explained variance = ', svd3.explained_variance_ratio_.sum())

# END YOUR CODE

Reduce dimension to 1000 and explained variance =  0.6211811379502958
Reduce dimension to 2000 and explained variance =  0.8167061772367108
Reduce dimension to 3000 and explained variance =  0.9176758858740324


In [19]:
word_vectors = word_vectors3
word_vectors.shape

(6690, 3000)

# 新的詞向量 + Group mean vector: 測試

In [20]:
excluded_flags = [
    'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
    'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
    'Caa', 'Cab', 'Cba', 'Cbb',
    'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
    'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
    'SPCHANGECATEGORY', 'WHITESPACE'
]

In [21]:
train_svd_vectors = {}
for index, pairs in train_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    train_svd_vectors[index] = vector
    

In [22]:
test_svd_vectors = {}
for index, pairs in test_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    test_svd_vectors[index] = vector

In [23]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(train_svd_vectors.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)

In [24]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [25]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_svd_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [26]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 65, '遊戲': 9, '財經': 8, '旅遊': 5, '科技': 4, '農業': 4})
predict 財經 :  Counter({'財經': 64, '科技': 22, '農業': 15, '體育': 7, '旅遊': 6, '遊戲': 6})
predict 科技 :  Counter({'科技': 56, '體育': 15, '財經': 13, '遊戲': 8, '農業': 7, '旅遊': 6})
predict 旅遊 :  Counter({'旅遊': 62, '農業': 10, '財經': 6, '科技': 6, '遊戲': 5, '體育': 2})
predict 農業 :  Counter({'農業': 61, '旅遊': 9, '體育': 3, '科技': 3, '遊戲': 3, '財經': 2})
predict 遊戲 :  Counter({'遊戲': 69, '旅遊': 10, '科技': 8, '財經': 7, '體育': 6, '農業': 2})


In [24]:
# from collections import Counter

# for group, ids in classification.items():
#     counter = Counter([test_classes[id] for id in ids])
#     print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 61, '遊戲': 10, '旅遊': 9, '財經': 8, '科技': 5, '農業': 4})
predict 財經 :  Counter({'財經': 62, '科技': 25, '農業': 16, '體育': 8, '遊戲': 8, '旅遊': 7})
predict 科技 :  Counter({'科技': 51, '體育': 15, '財經': 14, '農業': 9, '遊戲': 9, '旅遊': 8})
predict 旅遊 :  Counter({'旅遊': 58, '農業': 11, '科技': 5, '財經': 4, '體育': 2, '遊戲': 2})
predict 農業 :  Counter({'農業': 58, '旅遊': 7, '體育': 4, '財經': 4, '遊戲': 4, '科技': 2})
predict 遊戲 :  Counter({'遊戲': 67, '科技': 11, '旅遊': 9, '體育': 8, '財經': 8, '農業': 1})
